
# Building Agents with the Foundation Models API

This short notebooks is intended to help you start building agents with models available through the foundation models API (which you can use with your hackathon credits) and popular agent-building frameworks. It will also show how to use your own API keys to access outside models, if you so choose.

## Getting Started—Notes

- We recommend using `databricks-llama-4-maverick` as the base model for building your agents—it is a fast and highly capable model. If you encounter any compatibility issues, you can try `databricks-meta-llama-3-3-70b-instruct` as an alternative, or configure an [external model](#bring-your-own-model).
- Though Anthropic's Claude models are listed in the Serving menu, they are not currently usable in trial accounts. If you add your credit card information, you can use the Claude models via Databricks. This will still consume your credits and not charge your card directly (though you will be charged if you consume all of your trial credits)


## LlamaIndex

- See the [llamaindex docs](https://docs.llamaindex.ai/en/stable/examples/llm/databricks/) for more details on getting started with LlamaIndex and Databricks


### Setup: Install Dependencies

We will install the llamaindex package as well as the `llama-index-llms-databricks` package, which lets us configure and use models from Databricks model serving.

In [0]:
%pip install -U llama-index llama-index-llms-databricks mlflow
%restart_python

In [0]:
# Configure your personal access token
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
tmp_token = w.tokens.create(comment="for model serving", lifetime_seconds=1200)

### Set up your Databricks model with LlamaIndex

In [0]:
from llama_index.llms.databricks import Databricks

llm = Databricks(
    model="databricks-llama-4-maverick",
    api_key=tmp_token.token_value,
    api_base=f"{w.config.host}/serving-endpoints/"
)

### Query your Model

You can query the llamaindex-configured Databricks model as follows:

In [0]:
# Completion
llm.complete("Hello, world")

In [0]:
# Chat
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a helpful assistant"
    ),
    ChatMessage(role="user", content="Hello, world"),
]
llm.chat(messages)

### Create a LlamaIndex Agent

We can use the llm we configured to create a [Llamaindex Agent](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/).

Note that the Databricks model is compatible with `ReActAgent` but not with `FunctionAgent`. You can still easily use function calling with `ReActAgent`.

In [0]:
from llama_index.core.agent.workflow import ReActAgent


# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b


# Create an agent workflow with our calculator tool
agent = ReActAgent(
    tools=[multiply],
    llm=llm,
    system_prompt="You are a helpful assistant that can multiply two numbers. Always explain your answer in the final output. Tell the user if you called on any tools.",
)

response = await agent.run("What is 1234 * 4567? Tell me how you got the answer.")

response.response

You can find more integrations for LlamaIndex, including for [Databricks Vector Search](https://llamahub.ai/l/vector_stores/llama-index-vector-stores-databricks?from=), at [llamahub.ai](https://llamahub.ai/). This is the best source for finding tools to connect your agent to e.g. web search tools, vector stores, additional LLM providers, and more. For example, here's how we can enable DuckDuckGo web search for our agent.

We will also trace this interaction with mlflow, making it easy to keep track of all the steps of the agent execution.

In [0]:
%pip install llama-index-tools-duckduckgo

In [0]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
import mlflow

mlflow.llama_index.autolog()

tool_spec = DuckDuckGoSearchToolSpec()
search_tool_list = tool_spec.to_tool_list()

agent = ReActAgent(
    tools=search_tool_list,
    llm=llm,
    system_prompt="You are a helpful assistant that can search the web. Answer questions for the user based on web searches.",
)

response = await agent.run("What is the largest city in Croatia?")

response

This should be enough to get you started building agents with LlamaIndex and Databricks model serving. As a quick recap:
- Use the LlamaIndex `ReActAgent` to get started. The `FunctionAgent` does not currently work with Databricks model serving.
- Get additional tools and connections for your agent from llamahub.ai
- Use Databricks model serving models with LlamaIndex via the `llama-index-llms-databricks` package
- Trace your LlamaIndex interactions with `mlflow.llama_index.autolog()`

## LangGraph

You can learn more about the Databricks langchain/langgraph integration [here](https://python.langchain.com/docs/integrations/providers/databricks/).

### Setup: Install Dependencies

In [0]:
%pip install -U -qqqq databricks-langchain langgraph langchain mlflow 
%restart_python

### Set up your Databricks model with LangChain

In [0]:
from databricks_langchain import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

w = WorkspaceClient()

os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

### Query your Model

In [0]:
llm.invoke("hello, world")

### Create a LangGraph Agent

Note that `databricks-llama-4-maverick` is not currently compatible with multi-turn tool calling, so we will instead use `databricks-meta-llama-3-3-70b-instruct`, which will work appropriately with LangGraph.

In [0]:
from langgraph.prebuilt import create_react_agent

llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")

def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b

agent = create_react_agent(
    model=llm,  
    tools=[multiply],  
    prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is 1267 times 842"}]}
)


As we did with the LlamaIndex agent, we will now:

- Add web search to our LangGraph agent
- trace our agent execution with mlflow

In [0]:
from langchain_community.tools import DuckDuckGoSearchRun
import mlflow

mlflow.langchain.autolog()

search = DuckDuckGoSearchRun()

agent = create_react_agent(
    model=llm,  
    tools=[search],  
    prompt="You are a helpful assistant that can search the web. Answer questions for the user based on web searches."  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "What is the largest city in Croatia?"}]}
)

## Bring your own Model

If you want to use models not available through Databricks model serving, or otherwise want to use your own API keys with external model providers, you can do so in a couple of different ways. This might be a good idea if you are running into any compatiblity issues with various agent-building libraries and integrations. OpenAI models in particular tend to be highly compatible with external tools, and models such as `gpt-4o` and `gpt-4o-mini` are relatively inexpensive and still quite capable.

1. **Use the model provider's client directly.** This is the same as if you were using the model provider locally. You will need to install the relevant libraries and supply your own API key. e.g.

```python
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = <your_api_key>

client = OpenAI()

client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "user",
            "content": "Hello, World"
        }
    ]
)
```

2. **Configure the model as an external model in Databricks model serving**: See [here](https://docs.databricks.com/aws/en/generative-ai/external-models/) for details.
    1. Navigate to the `Serving` tab in the left sidebar and click "Create Serving Endpoint"
    2. Name the endpoint whatever you want in the "Name" field
    3. Click "Select an entity" under "Entity Details." Leave "Foundation models" selected.
    4. From the "Select a foundation model" dropdown, under "External model providers," select your model provider (e.g. OpenAI)
    5. Fill out the required details, including the API key and provider model and, optionally, any [AI Gateway configurations](https://docs.databricks.com/aws/en/ai-gateway/) you would like.
    6. Query the model in the same way as demonstrated above, using your Databricks workspace token and Databricks model serving integrations


### External Models—quick example

Suppose you have created a serving endpoint for OpenAI's gpt-4o-mini using the steps detailed above. Let's use this with LangGraph.

In [0]:
from langgraph.prebuilt import create_react_agent
import os

w = WorkspaceClient()

os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="4omini") # the name given to the endpoint in step 2 above

def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b

agent = create_react_agent(
    model=llm,  
    tools=[multiply],  
    prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is 1267 times 842"}]}
)